# Modifying Excel Files, Web Scrapping and Data Cleaning

## Web Scrapping

In [2]:
import pandas as pd
import re
import requests
from xls2xlsx import XLS2XLSX
from os import listdir
from bs4 import BeautifulSoup

In [3]:
# Here, we are going to convert the datafiles from .xls (old versions) to .xlsx to be read by pandas.
files = sorted(listdir('/Users/federicokurten/Desktop/Projects/Diabetes_Prevalence/Diabetes_per_Country'))
files = files[1:]
for i in files: 
    xls = XLS2XLSX(f'/Users/federicokurten/Desktop/Projects/Diabetes_Prevalence/Diabetes_per_Country/{i}')
    xls.to_xlsx(f'/Users/federicokurten/Desktop/Projects/Diabetes_Prevalence/Diabetes_per_Country_copy/{i}x')

In [4]:
# Now we have to modify the xlsx files to a cleaner form using pandas. We will do this by importing one data 
# file and use it as a global df for later to be concatenated with the data of the other countries.

global_df = pd.read_excel('Diabetes_per_Country_copy/Afghanistan-diabetes-report.xlsx')
global_df = global_df.transpose()
global_df = global_df.loc[:,[1,9,22,24]]
global_df.columns = global_df.iloc[0]
global_df.columns = ['People with diabetes 2010', 'Deaths attributable to diabetes 2010', 
                     'Total diabetes-related health expenditure, USD million 2010', 
                     'Mean diabetes-related health expenditure per person, USD 2010']
global_df = global_df.drop(["At a glance", 2030, 2045]).reset_index()\
            .rename(columns={'index':'Year'})
global_df['People with diabetes 2010'] *= 1000 
global_df['People with diabetes 2019'] = global_df.iloc[1,1]
global_df['Deaths attributable to diabetes 2019'] = global_df.iloc[1,2]
global_df['Total diabetes-related health expenditure, USD million 2019'] = global_df.iloc[1,3]
global_df['Mean diabetes-related health expenditure per person, USD 2019'] = global_df.iloc[1,4]
global_df['Country'] = 'Afghanistan'
global_df = global_df.drop(1).drop(columns=['Year'])


# Having the global df we are going to retrieve the 3 letter ISO codes from each country to be included in the 
# dataframe using BeautifulSoup and the requests modules from the World Population Review webpage and generate 
# a .csv file.

url = requests.get('https://worldpopulationreview.com/country-rankings/country-codes')
soup = BeautifulSoup(url.text, 'html.parser')
code_table = soup.find('table', class_ = 'jsx-1487038798 table table-striped tp-table-body')

countries = []
country_code = []
for c in code_table.find_all('tbody'): 
    rows = c.find_all('tr')
    for row in rows:
        country_info = row.find_all('td')
        country,a,code,b = country_info
        countries.append(country.text)
        country_code.append(code.text)
        
df_codes = pd.DataFrame(data={'Country':countries, 'Code':country_code})
df_codes.to_csv('/Users/federicokurten/Desktop/Projects/Diabetes_Prevalence/Country_Codes.csv')
global_df['Country Code'] = country_code[0]
global_df = global_df.iloc[:, [8,9,0,1,2,3,4,5,6,7]]
display(global_df)

# Lastly, we are going to retrieve all the countries from the Diabetes Prevalence scrapped data to compare them 
# to the countries we retrieved from the World Population Review webpage as the country's name are not the same 
# as well as some other countries that are not included. 

files = sorted(listdir('/Users/federicokurten/Desktop/Projects/Diabetes_Prevalence/Diabetes_per_Country_copy'))
files = files[1:]
countries = [re.search(r'([^.-]{1,})', i).group() for i in files]
df_country = pd.DataFrame(data={'Country':countries})
df_country.to_csv('/Users/federicokurten/Desktop/Projects/Diabetes_Prevalence/Country_Order.csv')

,Country,Country Code,People with diabetes 2010,Deaths attributable to diabetes 2010,"Total diabetes-related health expenditure, USD million 2010","Mean diabetes-related health expenditure per person, USD 2010",People with diabetes 2019,Deaths attributable to diabetes 2019,"Total diabetes-related health expenditure, USD million 2019","Mean diabetes-related health expenditure per person, USD 2019"
0,Afghanistan,AFG,856500.0,19031,28.6,33,1090800.0,18630.3,182.7,167.5


After we cleaned the data, web scrapped the web to find the 3 letter ISO codes for each country, we now want to include the codes for each country and aggregate it into `global_df` that we just generated.

In [5]:
df_codes = pd.read_csv('Country_Codes_Clean.csv')
country_code = df_codes['Code'].tolist()

for i,code in zip(files[1:],country_code[1:]):
    country = re.search(r'([^.-]{1,})', i).group()
    df = pd.read_excel(f'Diabetes_per_Country_copy/{i}')
    df = df.transpose()
    df = df.loc[:,[1,9,22,24]]
    df.columns = df.iloc[0]
    df.columns = ['People with diabetes 2010', 'Deaths attributable to diabetes 2010', 
                         'Total diabetes-related health expenditure, USD million 2010', 
                         'Mean diabetes-related health expenditure per person, USD 2010']
    df = df.drop(["At a glance", 2030, 2045]).reset_index()\
                .rename(columns={'index':'Year'})
    df['People with diabetes 2010'] *= 1000 

    df['People with diabetes 2019'] = df.iloc[1,1]
    df['Deaths attributable to diabetes 2019'] = df.iloc[1,2]
    df['Total diabetes-related health expenditure, USD million 2019'] = df.iloc[1,3]
    df['Mean diabetes-related health expenditure per person, USD 2019'] = df.iloc[1,4]
    df['Country'] = country
    df['Country Code'] = code
    df = df.drop(1).drop(columns=['Year'])
    df = df.iloc[:, [8,9,0,1,2,3,4,5,6,7]]

    global_df = pd.concat([global_df, df], ignore_index=True)

global_df

,Country,Country Code,People with diabetes 2010,Deaths attributable to diabetes 2010,"Total diabetes-related health expenditure, USD million 2010","Mean diabetes-related health expenditure per person, USD 2010",People with diabetes 2019,Deaths attributable to diabetes 2019,"Total diabetes-related health expenditure, USD million 2019","Mean diabetes-related health expenditure per person, USD 2019"
0,Afghanistan,AFG,856500.0,19031,28.6,33,1090800.0,18630.3,182.7,167.5
1,Albania,ALB,102800.0,1358,26.8,261,237600.0,2448.1,155.1,652.8
2,Algeria,DZA,1632100.0,14078,264.2,162,1904700.0,12657.2,1515.0,795.4
3,Andorra,AND,4600.0,31,13.5,2970,6900.0,32.4,27.6,4005.5
4,Angola,AGO,223700.0,4791,14.7,66,532400.0,6987.7,282.3,530.2
...,...,...,...,...,...,...,...,...,...,...
212,Viet Nam,VNM,1646600.0,32505,101.8,62,3779600.0,30096.2,1220.1,322.8
213,Western Sahara,ESH,13500.0,NaN,NaN,NaN,----------------------------------------------...,-,-,-
214,Yemen,YEM,270000,3410,20,74,572700.0,5772.3,NaN,NaN
215,Zambia,ZMB,169000,6529,11.1,66,273800.0,8000.2,81.3,296.8


Having the country's diabetes information and the 3 letter codes, we can now merge the `global_df` to the excel file of population that was found on the internet.

In [6]:
df_population = pd.read_csv('Population.csv')

global_df = global_df.merge(df_population, how='left', on='Country Code')
global_df

,Country,Country Code,People with diabetes 2010,Deaths attributable to diabetes 2010,"Total diabetes-related health expenditure, USD million 2010","Mean diabetes-related health expenditure per person, USD 2010",People with diabetes 2019,Deaths attributable to diabetes 2019,"Total diabetes-related health expenditure, USD million 2019","Mean diabetes-related health expenditure per person, USD 2019",Country Name,2010,2019
0,Afghanistan,AFG,856500.0,19031,28.6,33,1090800.0,18630.3,182.7,167.5,Afghanistan,29185511.0,38041757.0
1,Albania,ALB,102800.0,1358,26.8,261,237600.0,2448.1,155.1,652.8,Albania,2913021.0,2854191.0
2,Algeria,DZA,1632100.0,14078,264.2,162,1904700.0,12657.2,1515.0,795.4,Algeria,35977451.0,43053054.0
3,Andorra,AND,4600.0,31,13.5,2970,6900.0,32.4,27.6,4005.5,Andorra,84454.0,77146.0
4,Angola,AGO,223700.0,4791,14.7,66,532400.0,6987.7,282.3,530.2,Angola,23356247.0,31825299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Viet Nam,VNM,1646600.0,32505,101.8,62,3779600.0,30096.2,1220.1,322.8,Vietnam,87967655.0,96462108.0
213,Western Sahara,ESH,13500.0,NaN,NaN,NaN,----------------------------------------------...,-,-,-,NaN,NaN,NaN
214,Yemen,YEM,270000,3410,20,74,572700.0,5772.3,NaN,NaN,"Yemen, Rep.",23154854.0,29161922.0
215,Zambia,ZMB,169000,6529,11.1,66,273800.0,8000.2,81.3,296.8,Zambia,13605986.0,17861034.0


## Data Cleaning

We can see that all the population values got in perfectly into `global_df`. We now have to data clean the df before doing any analysis. To do this, we have to see which values have got `NaN` values from the population information that got merged, as it was a Left Join, where only the values being in the Left df (`global_df`) will be kept while the countries that were not in the population information will become `NaN`.  

In [7]:
NaN_vals = global_df[(global_df['Country Name'].isnull()) & (global_df['2010'].isnull()) \
                     & (global_df['2019'].isnull())]
print(NaN_vals.shape)
NaN_vals

(11, 13)


,Country,Country Code,People with diabetes 2010,Deaths attributable to diabetes 2010,"Total diabetes-related health expenditure, USD million 2010","Mean diabetes-related health expenditure per person, USD 2010",People with diabetes 2019,Deaths attributable to diabetes 2019,"Total diabetes-related health expenditure, USD million 2019","Mean diabetes-related health expenditure per person, USD 2019",Country Name,2010,2019
5,Anguilla,AIA,500.0,NaN,NaN,NaN,----------------------------------------------...,-,-,-,NaN,NaN,NaN
44,Cook Islands,COK,800.0,4,0.5,678,----------------------------------------------...,-,-,-,NaN,NaN,NaN
70,French Guiana,GUF,14500.0,NaN,NaN,NaN,----------------------------------------------...,-,-,-,NaN,NaN,NaN
80,Guadeloupe,GLP,32500.0,NaN,NaN,NaN,----------------------------------------------...,-,-,-,NaN,NaN,NaN
122,Martinique,MTQ,30500.0,NaN,NaN,NaN,----------------------------------------------...,-,-,-,NaN,NaN,NaN
136,Netherlands Antilles,ANT,19900.0,NaN,NaN,NaN,----------------------------------------------...,-,-,-,NaN,NaN,NaN
143,Niue,NIU,0,0,0.1,1594,----------------------------------------------...,-,-,-,NaN,NaN,NaN
160,Reunion,REU,87200.0,NaN,NaN,NaN,----------------------------------------------...,-,-,-,NaN,NaN,NaN
189,Taiwan,TWN,815900.0,NaN,NaN,NaN,1228800.0,NaN,NaN,NaN,NaN,NaN,NaN
194,Tokelau,TKL,100.0,NaN,NaN,NaN,----------------------------------------------...,-,-,-,NaN,NaN,NaN


There 11 countries that were in the `global_df` and not in the `df_population`. There is another problem that can be seen in the `People with diabetes 2019` column. Some values are a conglomeration of `-` and therefore those countries without information about diabetes need to be removed from the analysis. We will see in the other column `People with diabetes 2010` as well if there are some other strange values appearing in that column as well.

We will first remove those weird values from the dataframe.

In [8]:
weird_vals = global_df.loc[5,'People with diabetes 2019']
countries_with_vals = global_df[(global_df['People with diabetes 2010'] == weird_vals) | \
                                (global_df['People with diabetes 2019'] == weird_vals)]
print(countries_with_vals.shape)
countries_with_vals

(15, 13)


,Country,Country Code,People with diabetes 2010,Deaths attributable to diabetes 2010,"Total diabetes-related health expenditure, USD million 2010","Mean diabetes-related health expenditure per person, USD 2010",People with diabetes 2019,Deaths attributable to diabetes 2019,"Total diabetes-related health expenditure, USD million 2019","Mean diabetes-related health expenditure per person, USD 2019",Country Name,2010,2019
5,Anguilla,AIA,500.0,NaN,NaN,NaN,----------------------------------------------...,-,-,-,NaN,NaN,NaN
44,Cook Islands,COK,800.0,4,0.5,678,----------------------------------------------...,-,-,-,NaN,NaN,NaN
48,Curaçao,CUW,----------------------------------------------...,-,-,-,19700.0,NaN,NaN,NaN,Curacao,148703.0,157441.0
66,Faroe Islands,FRO,----------------------------------------------...,-,-,-,2400.0,NaN,NaN,NaN,Faroe Islands,47803.0,48677.0
70,French Guiana,GUF,14500.0,NaN,NaN,NaN,----------------------------------------------...,-,-,-,NaN,NaN,NaN
78,Greenland,GRL,----------------------------------------------...,-,-,-,1300.0,NaN,NaN,NaN,Greenland,56905.0,56225.0
80,Guadeloupe,GLP,32500.0,NaN,NaN,NaN,----------------------------------------------...,-,-,-,NaN,NaN,NaN
122,Martinique,MTQ,30500.0,NaN,NaN,NaN,----------------------------------------------...,-,-,-,NaN,NaN,NaN
136,Netherlands Antilles,ANT,19900.0,NaN,NaN,NaN,----------------------------------------------...,-,-,-,NaN,NaN,NaN
143,Niue,NIU,0,0,0.1,1594,----------------------------------------------...,-,-,-,NaN,NaN,NaN


We can see that there are 15 countries with no Diabetes information, and we will proceed in removing them.

In [9]:
countries = countries_with_vals['Country'].to_list()
global_df = global_df.query(f'Country != {countries}').reset_index(drop=True)
global_df

,Country,Country Code,People with diabetes 2010,Deaths attributable to diabetes 2010,"Total diabetes-related health expenditure, USD million 2010","Mean diabetes-related health expenditure per person, USD 2010",People with diabetes 2019,Deaths attributable to diabetes 2019,"Total diabetes-related health expenditure, USD million 2019","Mean diabetes-related health expenditure per person, USD 2019",Country Name,2010,2019
0,Afghanistan,AFG,856500.0,19031,28.6,33,1090800.0,18630.3,182.7,167.5,Afghanistan,29185511.0,38041757.0
1,Albania,ALB,102800.0,1358,26.8,261,237600.0,2448.1,155.1,652.8,Albania,2913021.0,2854191.0
2,Algeria,DZA,1632100.0,14078,264.2,162,1904700.0,12657.2,1515.0,795.4,Algeria,35977451.0,43053054.0
3,Andorra,AND,4600.0,31,13.5,2970,6900.0,32.4,27.6,4005.5,Andorra,84454.0,77146.0
4,Angola,AGO,223700.0,4791,14.7,66,532400.0,6987.7,282.3,530.2,Angola,23356247.0,31825299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,Venezuela (Bolivarian Republic of),VEN,1033700.0,8810,412.8,399,1403600.0,10649.0,NaN,NaN,"Venezuela, RB",28439942.0,28515829.0
198,Viet Nam,VNM,1646600.0,32505,101.8,62,3779600.0,30096.2,1220.1,322.8,Vietnam,87967655.0,96462108.0
199,Yemen,YEM,270000,3410,20,74,572700.0,5772.3,NaN,NaN,"Yemen, Rep.",23154854.0,29161922.0
200,Zambia,ZMB,169000,6529,11.1,66,273800.0,8000.2,81.3,296.8,Zambia,13605986.0,17861034.0


Having cleaned the weird values, we will now proceed in cleaning the countries where there is no information regarding population.

In [10]:
NaN_vals = global_df[(global_df['Country Name'].isnull()) & (global_df['2010'].isnull()) \
                     & (global_df['2019'].isnull())]
print(NaN_vals.shape)
NaN_vals

(1, 13)


,Country,Country Code,People with diabetes 2010,Deaths attributable to diabetes 2010,"Total diabetes-related health expenditure, USD million 2010","Mean diabetes-related health expenditure per person, USD 2010",People with diabetes 2019,Deaths attributable to diabetes 2019,"Total diabetes-related health expenditure, USD million 2019","Mean diabetes-related health expenditure per person, USD 2019",Country Name,2010,2019
176,Taiwan,TWN,815900.0,NaN,NaN,NaN,1228800.0,NaN,NaN,NaN,NaN,NaN,NaN


Now, the only country where there is no population data is Taiwan. We can therefore search through the web their respective population for 2010 and 2019.

Thanks to https://www.worldometers.info/world-population/population-by-country/ we can see that Taiwan in 2010 had a popuation of 23,187,551 and in 2019 they had a population of 23,816,775. We can now update that information in `global_df`.

In [11]:
global_df.loc[176,'2010'] = 23187551
global_df.loc[176,'2019'] = 23816775
global_df.query('Country == "Taiwan"')

,Country,Country Code,People with diabetes 2010,Deaths attributable to diabetes 2010,"Total diabetes-related health expenditure, USD million 2010","Mean diabetes-related health expenditure per person, USD 2010",People with diabetes 2019,Deaths attributable to diabetes 2019,"Total diabetes-related health expenditure, USD million 2019","Mean diabetes-related health expenditure per person, USD 2019",Country Name,2010,2019
176,Taiwan,TWN,815900.0,NaN,NaN,NaN,1228800.0,NaN,NaN,NaN,NaN,23187551.0,23816775.0


Done that, the last thing we need is to remove the column we are not going to be using in the analysis (Country Name), rearrange the columns and rename the `2010` and `2019` columns.

In [12]:
global_df = global_df.iloc[:, [0,1,11,12,2,6,3,7,4,8,5,9]]
global_df = global_df.rename(columns={'2010':'Population in 2010', '2019':'Population in 2019'})
global_df

,Country,Country Code,Population in 2010,Population in 2019,People with diabetes 2010,People with diabetes 2019,Deaths attributable to diabetes 2010,Deaths attributable to diabetes 2019,"Total diabetes-related health expenditure, USD million 2010","Total diabetes-related health expenditure, USD million 2019","Mean diabetes-related health expenditure per person, USD 2010","Mean diabetes-related health expenditure per person, USD 2019"
0,Afghanistan,AFG,29185511.0,38041757.0,856500.0,1090800.0,19031,18630.3,28.6,182.7,33,167.5
1,Albania,ALB,2913021.0,2854191.0,102800.0,237600.0,1358,2448.1,26.8,155.1,261,652.8
2,Algeria,DZA,35977451.0,43053054.0,1632100.0,1904700.0,14078,12657.2,264.2,1515.0,162,795.4
3,Andorra,AND,84454.0,77146.0,4600.0,6900.0,31,32.4,13.5,27.6,2970,4005.5
4,Angola,AGO,23356247.0,31825299.0,223700.0,532400.0,4791,6987.7,14.7,282.3,66,530.2
...,...,...,...,...,...,...,...,...,...,...,...,...
197,Venezuela (Bolivarian Republic of),VEN,28439942.0,28515829.0,1033700.0,1403600.0,8810,10649.0,412.8,NaN,399,NaN
198,Viet Nam,VNM,87967655.0,96462108.0,1646600.0,3779600.0,32505,30096.2,101.8,1220.1,62,322.8
199,Yemen,YEM,23154854.0,29161922.0,270000,572700.0,3410,5772.3,20,NaN,74,NaN
200,Zambia,ZMB,13605986.0,17861034.0,169000,273800.0,6529,8000.2,11.1,81.3,66,296.8


Now that all the data frame is cleaned up, we are going to be exporting this data to be used in Tableau.

In [12]:
global_df.to_csv('/Users/federicokurten/Desktop/Projects/Diabetes_Prevalence/Global_data_cleaned.csv')